In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
# from imbens.ensemble import BalancedRandomForestClassifier

In [8]:
# 加载数据
# file_path = 'E:\竞赛\服创赛-A08\data\欺诈手段特征集.xlsx'
# data = pd.read_excel(file_path)

# file_path = "E:\竞赛\服创赛-A08\data\\analyzed_data_3.csv"
file_path = "E:\竞赛\服创赛-A08\data\process_data.csv"
data = pd.read_csv(file_path)


# 展示数据的前几行以及基本信息
data.head(), data.info(), data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 82 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   个人编码               16000 non-null  float64
 1   一天去两家医院的天数         16000 non-null  int64  
 2   就诊的月数              16000 non-null  int64  
 3   月就诊天数_MAX          16000 non-null  int64  
 4   月就诊天数_AVG          16000 non-null  float64
 5   月就诊医院数_MAX         16000 non-null  int64  
 6   月就诊医院数_AVG         16000 non-null  float64
 7   就诊次数_SUM           16000 non-null  int64  
 8   月就诊次数_MAX          16000 non-null  int64  
 9   月就诊次数_AVG          16000 non-null  float64
 10  月统筹金额_MAX          16000 non-null  float64
 11  月统筹金额_AVG          16000 non-null  float64
 12  月药品金额_MAX          16000 non-null  float64
 13  月药品金额_AVG          16000 non-null  float64
 14  医院_就诊天数_MAX        16000 non-null  int64  
 15  医院_就诊天数_AVG        16000 non-null  float64
 16  医院_统筹金_MAX         160

(           个人编码  一天去两家医院的天数  就诊的月数  月就诊天数_MAX  月就诊天数_AVG  月就诊医院数_MAX  \
 0  3.521200e+14           0      6          7   5.666667           3   
 1  3.521200e+14           0      6          4   2.500000           2   
 2  3.521200e+14           8      6          8   6.166667           3   
 3  3.521200e+14           0      6          6   3.666667           2   
 4  3.521200e+14           0      6          5   4.333333           1   
 
    月就诊医院数_AVG  就诊次数_SUM  月就诊次数_MAX  月就诊次数_AVG  ...  药品在总金额中的占比  个人支付的药品占比  \
 0    2.166667        34          7   5.666667  ...    0.939194   0.004262   
 1    1.333333        15          4   2.500000  ...    0.955626   0.002982   
 2    2.166667        45          9   7.500000  ...    0.783610   0.000332   
 3    1.833333        23          6   3.833333  ...    0.458649   0.000184   
 4    1.000000        26          5   4.333333  ...    0.983726   0.000316   
 
    检查总费用在总金额占比  个人支付检查费用占比  治疗费用在总金额占比  个人支付治疗费用占比  BZ_民政救助  BZ_城乡优抚  是否挂号  \
 0     0.05

In [9]:
# 删除高度相关的特征（>0.9）--可选
# data = data.drop(columns=['月就诊天数_MAX', '月就诊天数差_调整','月就诊医院数差','月统筹金额差','月药品金额差_调整',
#                           '医院_就诊天数差','月就诊医院数_MAX','医院_统筹金差_调整', '医院_药品差_调整' ])

# data.head()

In [10]:
# 分离特征和目标变量
X = data.drop('RES', axis=1)
y = data['RES']

# 分离类别
data_majority = data[data.RES == 0]
data_minority = data[data.RES == 1]

In [30]:
# 定义一个函数，用于单独训练和返回模型
def get_specific_ratio_model(X, y, specific_ratio):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    final_model = None

    for train_index, test_index in skf.split(X, y):
        # 分割数据
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # 分离多数和少数类
        X_train_majority = X_train[y_train == 0]
        X_train_minority = X_train[y_train == 1]

        # 执行欠采样
        majority_sample_size = int(len(X_train_minority) * specific_ratio)
        X_train_majority_downsampled = resample(X_train_majority, 
                                                replace=False,
                                                n_samples=majority_sample_size,
                                                random_state=123)
        y_train_majority_downsampled = y_train.loc[X_train_majority_downsampled.index]

        # 合并数据
        X_train_downsampled = pd.concat([X_train_majority_downsampled, X_train_minority])
        y_train_downsampled = pd.concat([y_train_majority_downsampled, y_train[y_train == 1]])

        # 创建并训练模型
        RF_1 = RandomForestClassifier(n_estimators=100, random_state=123, n_jobs=-1,max_depth=15)
        rf_model = RF_1.fit(X_train, y_train)
        # rf_model = BalancedRandomForestClassifier(n_estimators=100,random_state=42,
        #                                           sampling_strategy="all",
        #                                           replacement=True,
        #                                           bootstrap=False,)
        # rf_model.fit(X_train, y_train)

        # 更新最后一个模型
        final_model = rf_model

        # 设置阈值为0.6
        threshold = 0.5
        
        # 以概率形式获取预测结果
        y_probs = rf_model.predict_proba(X_test)[:, 1] # 获取正类的概率
        
        # 应用阈值
        y_pred_thresh = np.where(y_probs > threshold, 1, 0)
        # 生成分类报告
        classification_report_06 = classification_report(y_test, y_pred_thresh)
        
        print(classification_report_06)

    return final_model,X_test,y_test

In [31]:
# 获取比例为n:1的模型(提高比例会略微提高AUC，但不显著)
specific_ratio = 6
RF_1,X_test,y_test = get_specific_ratio_model(X, y, specific_ratio)

# 输出比例为4的模型
RF_1

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3042
           1       0.87      0.34      0.48       158

    accuracy                           0.96      3200
   macro avg       0.92      0.67      0.73      3200
weighted avg       0.96      0.96      0.96      3200

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3042
           1       0.76      0.30      0.43       158

    accuracy                           0.96      3200
   macro avg       0.86      0.65      0.71      3200
weighted avg       0.95      0.96      0.95      3200

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3041
           1       0.75      0.36      0.49       159

    accuracy                           0.96      3200
   macro avg       0.86      0.68      0.73      3200
weighted avg       0.96      0.96      0.96      3200

              preci

RandomForestClassifier(max_depth=15, n_jobs=-1, random_state=123)

In [32]:
from joblib import dump

# 保存模型
model_filename = 'E:\竞赛\服创赛-A08\\model\\random_forest_model.joblib'
dump(RF_1, model_filename)

print(f"模型已保存为 {model_filename}")

模型已保存为 E:\竞赛\服创赛-A08\model\random_forest_model.joblib


In [33]:
# 划分训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# 设置阈值为0.6
threshold = 0.6

# 以概率形式获取预测结果
y_probs = RF_1.predict_proba(X_test)[:, 1] # 获取正类的概率

# 应用阈值
y_pred_thresh = np.where(y_probs > threshold, 1, 0)

# 生成分类报告
classification_report_06 = classification_report(y_test, y_pred_thresh)

print(classification_report_06)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3041
           1       0.88      0.26      0.41       159

    accuracy                           0.96      3200
   macro avg       0.92      0.63      0.69      3200
weighted avg       0.96      0.96      0.95      3200



In [34]:
from sklearn.metrics import roc_auc_score, roc_curve
y_pred_proba = RF_1.predict_proba(X_test)[:, 1]

# 计算AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

AUC: 0.9135680293845744


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option("display.max_columns",100)
pd.set_option('display.max_rows',100)

from pylab import mpl 
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 为了显示负号正常


correlation_matrix = data.corr()


plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Features')
plt.show()

In [ ]:
high_corr_pairs = []

for col in correlation_matrix.columns:
    for row in correlation_matrix.index:
        if (correlation_matrix.loc[row, col] > 0.9) and (col != row):
            # Check if the pair is already added
            if not {(row, col), (col, row)}.intersection(set(high_corr_pairs)):
                high_corr_pairs.append((row, col))

high_corr_pairs

In [ ]:
# 获取特征重要性
feature_importances = RF_1.feature_importances_

# 排序特征重要性
sorted_idx = np.argsort(feature_importances)[::-1]

# 取出前二十个重要特征
feature_importances = np.array(X.columns)[sorted_idx][:]

feature_importances

In [ ]:
import shap

In [ ]:
# 创建一个解释器
explainer = shap.TreeExplainer(RF_1)

# 计算SHAP值
shap_values = explainer.shap_values(X)

In [ ]:
# 可视化摘要图
feature_names = X.columns.tolist()
shap.summary_plot(shap_values, X, feature_names=feature_names)


SHAP 值的条形图，展示了在一个分类模型中各个特征对于模型输出的平均影响。在这种图表中，每个特征的 SHAP 值都被绘制为一个条形，长度表示该特征对模型输出的平均影响大小。

图中的特征被分为两类：Class 1（蓝色条形）和 Class 0（红色条形果。每个条形的长度表示该特征对应类别预测结果的平均影响程度，即平均 SHAP 值的大小。一个特征的 SHAP 值越高，它对模型输出的影响就越大。

具体来说，图表上的一些特征，如“药品在总金额中的占比”，对 Class 0（红色）有较大的正向影响，而对 Class 1（蓝色）则影响较小。相反，如果就诊次数“标准化SUM”这个特征，它对 Class 1 的影响较大，而对 Class 0 影响较小。

在解释这个图的时候，需要注意的是：

- 如果条形向右延伸，则意味着该特征的增加会增加模型输出该类的概率。
- 如果条形向左延伸（如果有的话），则意味着该特征的增加会降低模型输出该类的概率。
- 条形的长度代表了特征值变化对模型预测结果值变化带来的影响分布，但在这张简化的条形图中，我们主要看的是平均影响。

In [ ]:
# 可视化力度图
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], X.iloc[0])


In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][1], X.iloc[1])

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1], X)
# shap.force_plot(explainer.expected_value[0], shap_values[0], X)

In [ ]:
shap_values

SHAP 值的力度图（force plot），它是一个交互式的可视化，用来表示单个预测的解释。展示了一个模型输出的实际预测值（f(x)）是如何由不同特征的 SHAP 值相加而来的。

在力度图中：

- **基线值（Base value*：这是模型在没有任何特征输入时的平均例子中，基线值是 0.，这可以视为模型的“起点”。

- **输出值（f(）**：这是模型的实际预测值，在本例中为 0.94。

- **红色/蓝色条形**：每个条形代表一个特征对预测结果的贡献（SHAP 值）。红色条形表示该特征将预测题中通常是正类），蓝色条形则表别（通常是负类）。

- **特征值**：条形的长度表示该特征对模型预测结果的贡献大小。更长的条形表示影响更大。该图显示“药品在就诊次数_中的占比”和“标准化SUM”是主要影响因素，且它们对预测结果的影响方向相反。

在这个特定的预测实例中，我们可以看到：

- “药品在总金额中的占比”对预测结果有很大的负面影响（大红色条形），SHAP 值为 -0.9392就诊次数_SUM测结果。
- “标准化SUM”有一个正面影响（大蓝色条形），SHAP 值为 2.167，这提高了预测结果。

最终，所有这些特征的影响加在一起，导致模型的输出是 0.94，这是哪些因素在模型的决策过程中起了主导作用。

In [ ]:
X.columns

In [ ]:
# 遍历所有特征并绘制依赖图
for feature in X.columns:
    shap.dependence_plot(feature, shap_values[1], X, display_features=X)
    plt.show()  # 显示图表

这张图是一个 SHAP 依赖图（dependence plot），它展示了单个特征（在此例中为“就诊次数_SUM”）的 SHAP 值与该特征的实际值之间的关系。

在这张图中：

x轴就诊次数_SUM化SUM”的实际值。
y轴：显示该特征对模型输出影响的 SHAP 值。
颜色条：代有直接标注），颜色深浅表示该特征的数值大小。
就诊次数_SUM“标准化SUM”在不同值时对模型预测的影响（即SHAP值）。点的颜色变化表示一个额外的特征对每个点的SHA有影响，这可能表明两个特征之间存在某种交互作用。图中的点集合呈现出一定的模式，可能暗示着特定特征值范围内模型预测的影响增加或减少。

从就诊次数_SUM，当“标准化SUM”的值较小（靠近x轴的左侧）时，SHAP就诊次数_SU，随着“标准化SUM”值的增加，SHAP 值先是增加后减少，然后再次增加，表明了该特征与预测目标之间可能存在非线性关系。

颜色变化显示了第二个特征（颜就诊次数_SUM如何影响“标准化SUM”对模型输出的影响。例如，某些颜色的点主要集中在 SHAP 值较高或较低的区域，这可能表明当第就诊次数_SUM变化时，“标准化SUM一步的特征工程或模型调整。

In [ ]:
# 决策图
shap.decision_plot(explainer.expected_value[0], shap_values[1][9], X.iloc[0])

In [ ]:
from sklearn.decomposition import PCA

# 使用 PCA 进行维度降低
shap_embedded = PCA(n_components=2).fit_transform(shap_values[1])

# 绘制嵌入图
plt.scatter(shap_embedded[:, 0], shap_embedded[:, 1], c=y)
plt.colorbar()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
from joblib import dump

# 保存模型
model_filename = 'E:\竞赛\服创赛-A08\\model\\random_forest_model.joblib'
dump(RF_1, model_filename)

print(f"模型已保存为 {model_filename}")